In [1]:
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from IPython.display import clear_output
import time
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os

In [2]:
#Open ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("https://www.amazon.in/s?k=Dune&i=stripbooks")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\harik\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-2-a9184be2b85f>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Low and High Resolution Book Image

In [11]:
def download_image(book_name, author="", num_images=5):
    
    query = book_name + " " + author
    page_url = f"https://www.amazon.in/s?k={query.strip()}&i=stripbooks"
    print("Searching for:", query)
    driver.get(page_url)
    print("Done!")
    items = driver.find_elements(By.CLASS_NAME, "s-result-item")
    images = driver.find_elements(By.CLASS_NAME, "s-image")

    i=0
    match = 0
    while True:
        if "Sponsored" in items[i].text or not book_name.lower() in items[i].text.lower():
            i+=1
            pass
        else:
            match+=1
            link1 = images[i-1].get_attribute("src")
            print("link1:", link1)
            images[i-1].click()
            driver.switch_to.window(driver.window_handles[-1])
            # time.sleep(1)
            try:
                driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
                link2 = driver.find_elements(By.ID, "igImage")[0].get_attribute("src")
                print("link2:", link2)
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
            except:
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
                
            #Save images using link
            response = requests.get(link1)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            print(f"Image shape is {w}x{h}")
            img_path = os.path.join(os.getcwd(), "img")
            file_name_low = f"{book_name}_{match}_{w}x{h}.jpg"
            img.save(os.path.join(img_path, "low", file_name_low))

            #Higher resolution
            response = requests.get(link2)
            img2 = Image.open(BytesIO(response.content))
            w, h = img2.size
            print(f"Image shape of higher resultion is {w}x{h}")
            file_name_high = f"h_{book_name}_{match}_{w}x{h}.jpg"
            img2.save(os.path.join(img_path, "high", file_name_high))
            i+=1
            clear_output()
        if match==num_images or i==10:
            break
    

In [12]:
download_image("Emma", "Jane Austen", num_images=3)


## Other Features

In [13]:
cd 

'c:\\Users\\harik\\Desktop\\hari31416.github.io\\Projects\\Book_Library\\Python'

In [15]:
df = pd.read_csv("goodreads_library_export.csv")

In [ ]:
name = " ="1849511268""

In [22]:
df["ISBN"] = df['ISBN'].apply(lambda x: x.replace("\"", "").replace("=", ""))

In [25]:
df["ISBN13"]=df['ISBN13'].apply(lambda x: x.replace("\"", "").replace("=", ""))

In [ ]:
df.to

In [34]:
df.columns

Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Recommended For', 'Recommended By', 'Owned Copies',
       'Original Purchase Date', 'Original Purchase Location', 'Condition',
       'Condition Description', 'BCID'],
      dtype='object')

In [40]:
df[['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year']].to_csv("books_info.csv", index=False)